### requirements for the following codings


In [1]:
### packages required 
!pip install fair-esm 
!pip install torch
!pip install tensorflow
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.0 MB/s 


### peptide embeddings with esm2_t6_8M_UR50D pretrained models
6 layers, 8M parameters, dataset: UR50/D 2021_04, embedding dimension: 320
mode download URL: https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt

In [2]:
def esm_embeddings(peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you conputer might automatically kill the job.
  import torch
  import esm
  import collections
  # load the model
  # NOTICE: if the model was not downloaded in your local environment, it will automatically download it.
  model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
  batch_converter = alphabet.get_batch_converter()
  model.eval()  # disables dropout for deterministic results

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      results = model(batch_tokens, repr_layers=[6], return_contacts=True)  
  token_representations = results["representations"][6]

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  return embeddings_results

### data loading and embeddings (main dataset)

In [3]:
import numpy as np
import pandas as pd

In [4]:
# training dataset loading
dataset = pd.read_excel('AHT_train.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 

embeddings_results = pd.DataFrame()
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('AHT_train_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_train = dataset['label']
y_train = np.array(y_train) # transformed as np.array for CNN model

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t6_8M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D-contact-regression.pt


In [5]:
# test dataset loading
dataset = pd.read_excel('AHT_test.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
embeddings_results = pd.DataFrame()
# embedding all the peptide one by one
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('AHT_test_esm2_t6_8M_UR50D_unified_320_dimension.csv')


# loading the y dataset for model development 
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model

In [6]:
# assign the dataset 
X_train_data_name = 'AHT_train_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'AHT_test_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

In [7]:
# check the dimension of the dataset before model development
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1826, 320)
(772, 320)
(1826,)
(772,)


### Model architecture

In [9]:
def ESM_CNN(X_train, y_train, X_test, y_test):
  from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Conv1D
  from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, AveragePooling1D, MaxPooling1D
  from keras.models import Sequential,Model
  from keras.optimizers import SGD
  from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
  import keras
  from keras import backend as K
  inputShape=(320,1)
  input = Input(inputShape)
  x = Conv1D(128,(3),strides = (1),name='layer_conv1',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool1',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Conv1D(32,(3),strides = (1),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(64,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')
  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate 
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lrate = LearningRateScheduler(step_decay)

  # early stop setting
  early_stop = EarlyStopping(monitor='val_accuracy', patience = 40,restore_best_weights = True)

  # summary the callbacks_list
  callbacks_list = [ lrate , early_stop]

  model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                            epochs=200,callbacks=callbacks_list,batch_size = 8, verbose=1)
  return model, model_history

### 10-fold cross validation

In [16]:
#Implementing 10-fold cross validation
from sklearn.model_selection import KFold
k = 10 
kf = KFold(n_splits=k, shuffle = True, random_state=1)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []

for train_index , test_index in kf.split(y_train):
    X_train_CV , X_valid_CV = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
    y_train_CV , y_valid_CV = y_train.iloc[train_index] , y_train.iloc[test_index]
    model, model_history = ESM_CNN(X_train_CV, y_train_CV, X_valid_CV, y_valid_CV)
    # confusion matrix 
    predicted_class= []
    predicted_protability = model.predict(X_valid_CV,batch_size=1)
    for i in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_valid_CV    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    from sklearn.metrics import roc_auc_score
    AUC = roc_auc_score(y_valid_CV, predicted_protability[:,1])
    AUC_collecton.append(AUC)


Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


206/206 [==============================] - 5s 19ms/step - loss: 0.8573 - accuracy: 0.6166 - val_loss: 0.7405 - val_accuracy: 0.5191 - lr: 0.1000
Epoch 2/200
206/206 [==============================] - 2s 11ms/step - loss: 0.5763 - accuracy: 0.7042 - val_loss: 0.6444 - val_accuracy: 0.6230 - lr: 0.1000
Epoch 3/200
206/206 [==============================] - 2s 11ms/step - loss: 0.5486 - accuracy: 0.7115 - val_loss: 0.5922 - val_accuracy: 0.6667 - lr: 0.0600
Epoch 4/200
206/206 [==============================] - 2s 10ms/step - loss: 0.5135 - accuracy: 0.7395 - val_loss: 0.5745 - val_accuracy: 0.7377 - lr: 0.0600
Epoch 5/200
206/206 [==============================] - 2s 11ms/step - loss: 0.4945 - accuracy: 0.7559 - val_loss: 0.5572 - val_accuracy: 0.7268 - lr: 0.0600
Epoch 6/200
206/206 [==============================] - 2s 10ms/step - loss: 0.4545 - accuracy: 0.7718 - val_loss: 0.6294 - val_accuracy: 0.7377 - lr: 0.0360
Epoch 7/200
206/206 [==============================] - 2s 10ms/step - 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


206/206 [==============================] - 3s 11ms/step - loss: 0.8497 - accuracy: 0.6397 - val_loss: 0.6929 - val_accuracy: 0.4863 - lr: 0.1000
Epoch 2/200
206/206 [==============================] - 2s 10ms/step - loss: 0.5460 - accuracy: 0.7298 - val_loss: 0.6412 - val_accuracy: 0.6776 - lr: 0.1000
Epoch 3/200
206/206 [==============================] - 2s 10ms/step - loss: 0.4868 - accuracy: 0.7389 - val_loss: 0.5610 - val_accuracy: 0.7104 - lr: 0.0600
Epoch 4/200
206/206 [==============================] - 2s 10ms/step - loss: 0.4752 - accuracy: 0.7553 - val_loss: 0.6044 - val_accuracy: 0.6885 - lr: 0.0600
Epoch 5/200
206/206 [==============================] - 2s 10ms/step - loss: 0.4574 - accuracy: 0.7657 - val_loss: 0.6123 - val_accuracy: 0.7049 - lr: 0.0600
Epoch 6/200
206/206 [==============================] - 2s 10ms/step - loss: 0.4125 - accuracy: 0.7998 - val_loss: 0.5954 - val_accuracy: 0.6995 - lr: 0.0360
Epoch 7/200
206/206 [==============================] - 2s 10ms/step - 

...

206/206 [==============================] - 3s 12ms/step - loss: 1.0450 - accuracy: 0.6600 - val_loss: 0.6844 - val_accuracy: 0.5659 - lr: 0.1000
Epoch 2/200
206/206 [==============================] - 2s 10ms/step - loss: 0.5581 - accuracy: 0.7178 - val_loss: 0.6141 - val_accuracy: 0.7253 - lr: 0.1000
Epoch 3/200
206/206 [==============================] - 2s 11ms/step - loss: 0.5072 - accuracy: 0.7372 - val_loss: 0.5344 - val_accuracy: 0.7747 - lr: 0.0600
Epoch 4/200
206/206 [==============================] - 2s 10ms/step - loss: 0.4809 - accuracy: 0.7597 - val_loss: 0.5032 - val_accuracy: 0.7802 - lr: 0.0600
Epoch 5/200
206/206 [==============================] - 2s 10ms/step - loss: 0.4620 - accuracy: 0.7634 - val_loss: 0.5157 - val_accuracy: 0.7308 - lr: 0.0600
Epoch 6/200
206/206 [==============================] - 2s 10ms/step - loss: 0.4311 - accuracy: 0.7974 - val_loss: 0.4955 - val_accuracy: 0.7637 - lr: 0.0360
Epoch 7/200
206/206 [==============================] - 2s 10ms/step - 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


206/206 [==============================] - 3s 11ms/step - loss: 0.8522 - accuracy: 0.5103 - val_loss: 0.6916 - val_accuracy: 0.5275 - lr: 0.1000
Epoch 2/200
206/206 [==============================] - 2s 10ms/step - loss: 0.6672 - accuracy: 0.6004 - val_loss: 0.6496 - val_accuracy: 0.6429 - lr: 0.1000
Epoch 3/200
206/206 [==============================] - 2s 10ms/step - loss: 0.5990 - accuracy: 0.6715 - val_loss: 0.5857 - val_accuracy: 0.6978 - lr: 0.0600
Epoch 4/200
206/206 [==============================] - 2s 10ms/step - loss: 0.5526 - accuracy: 0.6983 - val_loss: 0.5481 - val_accuracy: 0.7088 - lr: 0.0600
Epoch 5/200
206/206 [==============================] - 2s 10ms/step - loss: 0.5266 - accuracy: 0.7220 - val_loss: 0.6725 - val_accuracy: 0.6923 - lr: 0.0600
Epoch 6/200
206/206 [==============================] - 2s 10ms/step - loss: 0.5174 - accuracy: 0.7141 - val_loss: 0.5127 - val_accuracy: 0.7637 - lr: 0.0360
Epoch 7/200
206/206 [==============================] - 2s 10ms/step - 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


206/206 [==============================] - 3s 12ms/step - loss: 0.7502 - accuracy: 0.5006 - val_loss: 0.6932 - val_accuracy: 0.5055 - lr: 0.1000
Epoch 2/200
206/206 [==============================] - 2s 11ms/step - loss: 0.6970 - accuracy: 0.5170 - val_loss: 0.7158 - val_accuracy: 0.4945 - lr: 0.1000
Epoch 3/200
206/206 [==============================] - 3s 15ms/step - loss: 0.6831 - accuracy: 0.5359 - val_loss: 0.6390 - val_accuracy: 0.6429 - lr: 0.0600
Epoch 4/200
206/206 [==============================] - 2s 11ms/step - loss: 0.6060 - accuracy: 0.6673 - val_loss: 0.6479 - val_accuracy: 0.6813 - lr: 0.0600
Epoch 5/200
206/206 [==============================] - 2s 11ms/step - loss: 0.6086 - accuracy: 0.6727 - val_loss: 0.8456 - val_accuracy: 0.5000 - lr: 0.0600
Epoch 6/200
206/206 [==============================] - 2s 10ms/step - loss: 0.5648 - accuracy: 0.7007 - val_loss: 0.6318 - val_accuracy: 0.6703 - lr: 0.0360
Epoch 7/200
206/206 [==============================] - 2s 11ms/step - 

In [17]:
from statistics import mean, stdev
print(mean(ACC_collecton),'±',stdev(ACC_collecton))
print(mean(BACC_collecton),'±',stdev(BACC_collecton))
print(mean(Sn_collecton),'±',stdev(Sn_collecton))
print(mean(Sp_collecton),'±',stdev(Sp_collecton))
print(mean(MCC_collecton),'±',stdev(MCC_collecton))
print(mean(AUC_collecton),'±',stdev(AUC_collecton))


0.7760133309313637 ± 0.025758007352775093
0.776555281078765 ± 0.025469196721485354
0.7749042812411983 ± 0.029337523994489023
0.7782062809163318 ± 0.042195951992903095
0.5522067009265982 ± 0.05138414095118637
0.8347310393191278 ± 0.029674962035681967


### model evaluation in test dataset

In [18]:
# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
model, model_history = ESM_CNN(X_train, y_train, X_test , y_test)
# confusion matrix 
predicted_class= []
predicted_protability = model.predict(X_test,batch_size=1)
for i in range(predicted_protability.shape[0]):
  index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
  predicted_class.append(index)
predicted_class = np.array(predicted_class)
y_true = y_test    
from sklearn.metrics import confusion_matrix
import math
# np.ravel() return a flatten 1D array
TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
ACC = (TP+TN)/(TP+TN+FP+FN)
ACC_collecton.append(ACC)
Sn_collecton.append(TP/(TP+FN))
Sp_collecton.append(TN/(TN+FP))
MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
MCC_collecton.append(MCC)
BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(y_test, predicted_protability[:,1])
AUC_collecton.append(AUC)

Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


229/229 [==============================] - 3s 11ms/step - loss: 0.9167 - accuracy: 0.6698 - val_loss: 0.7005 - val_accuracy: 0.5000 - lr: 0.1000
Epoch 2/200
229/229 [==============================] - 2s 10ms/step - loss: 0.5524 - accuracy: 0.7223 - val_loss: 0.5665 - val_accuracy: 0.7280 - lr: 0.1000
Epoch 3/200
229/229 [==============================] - 3s 14ms/step - loss: 0.4993 - accuracy: 0.7470 - val_loss: 0.4745 - val_accuracy: 0.7733 - lr: 0.0600
Epoch 4/200
229/229 [==============================] - 3s 12ms/step - loss: 0.4780 - accuracy: 0.7585 - val_loss: 0.4699 - val_accuracy: 0.7578 - lr: 0.0600
Epoch 5/200
229/229 [==============================] - 3s 11ms/step - loss: 0.4574 - accuracy: 0.7798 - val_loss: 0.5172 - val_accuracy: 0.7526 - lr: 0.0600
Epoch 6/200
229/229 [==============================] - 3s 11ms/step - loss: 0.4244 - accuracy: 0.7924 - val_loss: 0.4910 - val_accuracy: 0.7707 - lr: 0.0360
Epoch 7/200
229/229 [==============================] - 3s 11ms/step - 

In [19]:
print(ACC_collecton[0])
print(BACC_collecton[0])
print(Sn_collecton[0])
print(Sp_collecton[0])
print(MCC_collecton[0])
print(AUC_collecton[0])

0.8510362694300518
0.851718479963956
0.8362282878411911
0.8672086720867209
0.7027544182604775
0.9006349163735938


In [20]:
model.save('AHT_main_tensorflow_model',save_format = 'tf') 
!zip -r /content/AHT_main_tensorflow_model.zip /content/AHT_main_tensorflow_model

  adding: content/AHT_main_tensorflow_model/ (stored 0%)
  adding: content/AHT_main_tensorflow_model/variables/ (stored 0%)
  adding: content/AHT_main_tensorflow_model/variables/variables.index (deflated 64%)
  adding: content/AHT_main_tensorflow_model/variables/variables.data-00000-of-00001 (deflated 35%)
  adding: content/AHT_main_tensorflow_model/saved_model.pb (deflated 88%)
  adding: content/AHT_main_tensorflow_model/assets/ (stored 0%)
  adding: content/AHT_main_tensorflow_model/keras_metadata.pb (deflated 89%)
